In [ ]:
for iter in range(30000):                           # 順伝播
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])
    layers,loss = predict(sent)
    
    for layer_idx in reversed(range(len(layers))):  # 逆伝播
        layer = layers[layer_idx]
        target = sent[layer_idx-1]
        
        if(layer_idx > 0):
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = \
                layer['output_delta'].dot(decoder.transpose())
            
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + \
                    layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else:
            layer['hidden_delta'] = \
                layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
    # 重みを更新
    start -= layers[0]['hidden_delta'] * alpha / float()
    for layer_idx,layer in enumerate(layers[1:]):
        decoder -= np.outer(layers[layer_idx]['hidden'], \
                            layer['output_delta']) * alpha / float(len(sent))
        
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= \
            layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], \
                              layer['hidden_delta']) * alpha / float(len(sent))
        
    if(iter % 1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))